In [1]:
from google.colab import drive
drive.flush_and_unmount()
!rm -rf /content/drive/*
drive.mount('/content/drive')


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [2]:
import os
import numpy as np
import cv2
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

# Function to load all images from a folder
def load_images_from_folder(folder, img_size=(128, 128)):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            images.append(img)
    print(f"Loaded {len(images)} images from {folder}")
    return np.array(images)

# Paths to the benign and malignant directories
benign_dir="/content/drive/MyDrive/Kaggle_Filtered/benign_filtered"
malignant_dir="/content/drive/MyDrive/Kaggle_Filtered/malignant_filtered"

benign_images = load_images_from_folder(benign_dir)
malignant_images = load_images_from_folder(malignant_dir)

# Combine the images
images = np.concatenate((benign_images, malignant_images), axis=0)

# Normalize images (0-1)
images = images.astype(np.float32) / 255.0  # Ensure correct dtype for normalization

# Shuffle images
np.random.shuffle(images)

print(f"Total images loaded: {images.shape[0]}")




Loaded 1800 images from /content/drive/MyDrive/Kaggle_Filtered/benign_filtered
Loaded 1497 images from /content/drive/MyDrive/Kaggle_Filtered/malignant_filtered
Total images loaded: 3297


In [3]:
benign_images = benign_images.astype(np.float32) / 255.0
malignant_images = malignant_images.astype(np.float32) / 255.0

In [4]:
# Function to generate distinct synthetic masks for benign and malignant images
def generate_synthetic_masks(benign_images, malignant_images, threshold=128):
    masks = []

    # Generate benign masks (diffused or low-intensity masks)
    for image in benign_images:
        image_uint8 = (image * 255).astype(np.uint8)
        gray_image = cv2.cvtColor(image_uint8, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(gray_image, threshold - 30, 255, cv2.THRESH_BINARY)  # Lower threshold for benign
        mask = cv2.GaussianBlur(mask, (15, 15), 0)  # Diffuse edges for benign images
        mask = cv2.resize(mask, (128, 128))
        masks.append(mask / 255.0)

    # Generate malignant masks (higher intensity, more focused regions)
    for image in malignant_images:
        image_uint8 = (image * 255).astype(np.uint8)
        gray_image = cv2.cvtColor(image_uint8, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(gray_image, threshold + 30, 255, cv2.THRESH_BINARY)  # Higher threshold for malignant
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((5, 5), np.uint8))  # Enhance edges for malignant
        mask = cv2.resize(mask, (128, 128))
        masks.append(mask / 255.0)

    return np.array(masks).reshape(-1, 128, 128, 1)

# Generate synthetic masks for the benign and malignant images
lesion_masks = generate_synthetic_masks(benign_images, malignant_images)

# Combine benign and malignant images for final dataset
images = np.concatenate((benign_images, malignant_images), axis=0)

# Shuffle images and masks together
shuffled_indices = np.random.permutation(len(images))
images = images[shuffled_indices]
lesion_masks = lesion_masks[shuffled_indices]

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, lesion_masks, test_size=0.2, random_state=42)

# Print shapes of training and validation datasets
print(f"Training images shape: {X_train.shape}, Training masks shape: {y_train.shape}")
print(f"Validation images shape: {X_val.shape}, Validation masks shape: {y_val.shape}")

Training images shape: (2637, 128, 128, 3), Training masks shape: (2637, 128, 128, 1)
Validation images shape: (660, 128, 128, 3), Validation masks shape: (660, 128, 128, 1)


In [5]:
# 3. Define the updated SegNet model
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
def segnet(input_shape=(128, 128, 3)):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p4)

    # Decoder
    u1 = layers.UpSampling2D((2, 2))(c5)
    u1 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u1)
    u1 = layers.Concatenate()([u1, c4])

    u2 = layers.UpSampling2D((2, 2))(u1)
    u2 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u2)
    u2 = layers.Concatenate()([u2, c3])

    u3 = layers.UpSampling2D((2, 2))(u2)
    u3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u3)
    u3 = layers.Concatenate()([u3, c2])

    u4 = layers.UpSampling2D((2, 2))(u3)
    u4 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u4)
    u4 = layers.Concatenate()([u4, c1])

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(u4)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

In [6]:
# 4. Define Grey Wolf Optimizer
class GreyWolfOptimizer:
    def __init__(self, obj_function, lower_bound, upper_bound, population_size=5, iterations=20):
        self.obj_function = obj_function
        self.lower_bound = np.array(lower_bound)
        self.upper_bound = np.array(upper_bound)
        self.population_size = population_size
        self.iterations = iterations
        self.dimension = len(lower_bound)

    def optimize(self):
        wolves = np.random.uniform(self.lower_bound, self.upper_bound, (self.population_size, self.dimension))
        alpha, beta, delta = np.zeros(self.dimension), np.zeros(self.dimension), np.zeros(self.dimension)
        alpha_score, beta_score, delta_score = float("inf"), float("inf"), float("inf")

        for iteration in range(self.iterations):
            for i in range(self.population_size):
                score = self.obj_function(wolves[i])
                if score < alpha_score:
                    alpha_score, alpha = score, wolves[i]
                elif score < beta_score:
                    beta_score, beta = score, wolves[i]
                elif score < delta_score:
                    delta_score, delta = score, wolves[i]

            a = 2 - iteration * (2 / self.iterations)
            for i in range(self.population_size):
                r1, r2 = np.random.rand(), np.random.rand()
                A1, C1 = 2 * a * r1 - a, 2 * r2
                D_alpha = abs(C1 * alpha - wolves[i])
                X1 = alpha - A1 * D_alpha

                r1, r2 = np.random.rand(), np.random.rand()
                A2, C2 = 2 * a * r1 - a, 2 * r2
                D_beta = abs(C2 * beta - wolves[i])
                X2 = beta - A2 * D_beta

                r1, r2 = np.random.rand(), np.random.rand()
                A3, C3 = 2 * a * r1 - a, 2 * r2
                D_delta = abs(C3 * delta - wolves[i])
                X3 = delta - A3 * D_delta

                wolves[i] = (X1 + X2 + X3) / 3
                wolves[i] = np.clip(wolves[i], self.lower_bound, self.upper_bound)

        return alpha, alpha_score

In [7]:
# 5. Define Objective Function for Optimization
def objective_function(params):
    learning_rate = params[0]
    model = segnet()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss="binary_crossentropy", metrics=["accuracy"])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1, batch_size=16, verbose=0)
    return history.history['val_loss'][-1]

# 6. Run Grey Wolf Optimizer to Find Best Learning Rate
gwo = GreyWolfOptimizer(objective_function, lower_bound=[1e-5], upper_bound=[1e-2], population_size=5, iterations=4)
best_params, best_score = gwo.optimize()
print("Best Parameters Found:", best_params)
print("Best Validation Loss Achieved:", best_score)

Best Parameters Found: [0.00225497]
Best Validation Loss Achieved: 0.27273881435394287


In [8]:
segnet_model = segnet()
segnet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params[0]),
                     loss="binary_crossentropy", metrics=["accuracy"])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = segnet_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=16,
                           callbacks=[early_stopping])

Epoch 1/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 39s 201ms/step - accuracy: 0.6935 - loss: 0.7737 - val_accuracy: 0.7589 - val_loss: 0.3948
Epoch 2/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 36s 183ms/step - accuracy: 0.7301 - loss: 0.5261 - val_accuracy: 0.8354 - val_loss: 0.2773
Epoch 3/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 40s 178ms/step - accuracy: 0.8207 - loss: 0.3058 - val_accuracy: 0.8372 - val_loss: 0.2675
Epoch 4/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 30s 183ms/step - accuracy: 0.8176 - loss: 0.3065 - val_accuracy: 0.8351 - val_loss: 0.2654
Epoch 5/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 41s 181ms/step - accuracy: 0.8259 - loss: 0.2852 - val_accuracy: 0.8427 - val_loss: 0.2453
Epoch 6/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 41s 178ms/step - accuracy: 0.8377 - loss: 0.2585 - val_accuracy: 0.8403 - val_loss: 0.2501
Epoch 7/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 40s 176ms/step - accuracy: 0.8401 - loss: 0.2571 - val_accuracy: 0.8357 - val_loss: 0.2538
Epoch 8/50
165/165 ━━━━━━━━━━━━━━━━━━━━ 41s 177ms/step - accuracy: 0.8142 - loss: 0

In [9]:
best_model=segnet_model

In [10]:
import numpy as np
from sklearn.metrics import adjusted_rand_score, cohen_kappa_score

def jaccard_coef(y_true, y_pred, smooth=1e-10, threshold=0.5):
    """
    Calculate the Jaccard coefficient (Intersection over Union).

    Parameters:
    - y_true: Ground truth binary mask.
    - y_pred: Predicted binary mask.
    - smooth: Smoothing factor to avoid division by zero.
    - threshold: Threshold to convert predictions to binary.

    Returns:
    - Jaccard coefficient value.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Apply threshold to convert predictions to binary
    y_pred = (y_pred > threshold).astype(int)

    # Flatten the arrays
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()

    # Calculate intersection and union
    intersection = np.sum(y_true_f * y_pred_f)
    union = np.sum(y_true_f) + np.sum(y_pred_f) - intersection

    # Calculate the Jaccard coefficient (IoU)
    jac = (intersection + smooth) / (union + smooth)
    return jac


def dice_coef(y_true, y_pred, smooth=1e-10, threshold=0.5):
    """
    Calculate the Dice coefficient.

    Parameters:
    - y_true: Ground truth binary mask.
    - y_pred: Predicted binary mask.
    - smooth: Smoothing factor to avoid division by zero.
    - threshold: Threshold to convert predictions to binary.

    Returns:
    - Dice coefficient value.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Apply threshold to convert predictions to binary
    y_pred = (y_pred > threshold).astype(int)

    # Flatten the arrays
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()

    # Calculate intersection and sums
    intersection = np.sum(y_true_f * y_pred_f)
    sum_y_true = np.sum(y_true_f)
    sum_y_pred = np.sum(y_pred_f)

    # Calculate the Dice coefficient
    dice = (2. * intersection + smooth) / (sum_y_true + sum_y_pred + smooth)
    return dice


def rand_index(y_true, y_pred):
    """
    Calculate the Adjusted Rand Index.

    Parameters:
    - y_true: Ground truth binary mask.
    - y_pred: Predicted binary mask.

    Returns:
    - Adjusted Rand Index value.
    """
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    y_pred = (y_pred > 0.5).astype(int)

    # Calculate Adjusted Rand Index using scikit-learn
    ari = adjusted_rand_score(y_true, y_pred)
    return ari

import numpy as np
from sklearn.metrics import cohen_kappa_score

def cohen_kappa(y_true, y_pred, threshold=0.5):
    """
    Calculate Cohen's Kappa score.

    Parameters:
    - y_true: Ground truth binary mask.
    - y_pred: Predicted binary mask.
    - threshold: Threshold to convert predictions to binary.

    Returns:
    - Cohen's Kappa score value.
    """
    # Convert both y_true and y_pred to binary
    y_true = (np.array(y_true).flatten() > threshold).astype(int)
    y_pred = (np.array(y_pred).flatten() > threshold).astype(int)

    # Calculate Cohen's Kappa using scikit-learn
    kappa = cohen_kappa_score(y_true, y_pred)
    return kappa



In [11]:
# Predict on validation set
y_pred_val = best_model.predict(X_val)
# Calculate Jaccard coefficient (IoU)
jaccard_score = jaccard_coef(y_val, y_pred_val)
print(f"Jaccard Coefficient (IoU): {jaccard_score}")
dice_Score=dice_coef(y_val,y_pred_val)
print(f"Dice Coefficient : {dice_Score}")
cohens=cohen_kappa(y_val,y_pred_val)
print(f"Cohen's Kappa  : {cohens}")
rand=rand_index(y_val,y_pred_val)
print(f"Rand index  : {rand}")


21/21 ━━━━━━━━━━━━━━━━━━━━ 27s 614ms/step
Jaccard Coefficient (IoU): 0.8722704127426617
Dice Coefficient : 0.9317782375943072
Cohen's Kappa  : 0.7544245158443668


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/cluster/_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and binary values for target
  warnings.warn(msg, UserWarning)


Rand index  : 0.6085396321788812


In [12]:
best_model.save('/content/drive/MyDrive/best_model_segnet_Antcolony1_maskupdateGreyWolf.h5')  # Save the model to Google Drive or any desired directory

print("Model saved successfully!")

Model saved successfully!
